In [1]:
# v5 Ymir gets X-Y-Z so that 3 layers: 

# v4 : Ymir with varying inputs and outputs x-(x*y)-y network

from brian2 import *
import numpy as np
import logging, warnings

start_scope()

defaultclock.dt = 0.01*ms  

#prefs.codegen.target = 'cython'
set_device('runtime')
warnings.filterwarnings('ignore', category=RuntimeWarning)  # check error later
np.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)


decay_rate = 5*ms

def run_Ymir(inputs, y, z, taus1, taus2):

    x = len(inputs)
    xy_ind = x*y
    yz_ind = y*z


    if len(taus1) != xy_ind:
        print("not the right size for taus1 buddy")
        raise ValueError(f"Length of taus1 wrong .")
    
    if len(taus2) != yz_ind:
        print("not the right size for taus2 buddy")
        raise ValueError(f"Length of taus2 is wrong")

    input_neurons = NeuronGroup(x,'''
    dv/dt = -v/ decay_rate : volt                 
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    input_neurons.v = 0 * volt

    indices_input = []
    for i in range(0, x):
        indices_input.append(i)

    stim_input = SpikeGeneratorGroup(x, indices=indices_input, times= inputs * ms)

    syn_input = Synapses(stim_input, input_neurons[0:x], '''
    ''', on_pre='''
        v += 1.2 * volt
    ''')
    syn_input.connect(j='i') 

    ind_input_neurons = NeuronGroup(xy_ind,
    '''dv/dt = -v/ decay_rate : volt 
         
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    ind_input_neurons.v = 0 * volt

    input_range = []
    for i in range(0, x):
        for j in range(0, y):
            input_range.append(i)


    ind_input = Synapses(input_neurons[0:x], ind_input_neurons[0:xy_ind], '''
    ''', on_pre='''
        v += 0.55 * volt
    ''')
    ind_input.connect(i=input_range, j=[k for k in range(0, xy_ind)]) # look into what j is later


    stim_tau_hidden = SpikeGeneratorGroup(xy_ind, indices=[k for k in range(0, xy_ind)], times = taus1 * ms)

    syn_tau_hidden = Synapses(stim_tau_hidden, ind_input_neurons[0:xy_ind], '''
    ''', on_pre='''
        v += 0.55 * volt
    ''')
    syn_tau_hidden.connect(j='i')

    hidden_neurons = NeuronGroup(y,
    '''dv/dt = -v/ decay_rate : volt 
         
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    hidden_neurons.v = 0 * volt

    output_range = []
    for i in range(0, x):
        for j in range(0, y):
            output_range.append(j)

    syn_ind_hidden = Synapses(ind_input_neurons[0:xy_ind], hidden_neurons[0:y], '''
    ''', on_pre='''
        v += 1.2 * volt
    ''')

    syn_ind_hidden.connect(i=[k for k in range(0, xy_ind)], j=output_range)

    # last layer copy of above below but changed names same logic -- can functionalize later

    # stim_input = SpikeGeneratorGroup(x, indices=indices_input, times= inputs * ms)

    # syn_input = Synapses(stim_input, input_neurons[0:x], '''
    # ''', on_pre='''
    #     v += 1.2 * volt
    # ''')
    # syn_input.connect(j='i') 

    ind_hidden_neurons = NeuronGroup(yz_ind,
    '''dv/dt = -v/ decay_rate : volt 
         
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    ind_hidden_neurons.v = 0 * volt

    hidden_range = []
    for i in range(0, y):
        for j in range(0, z):
            hidden_range.append(i)


    syn_hidden_output = Synapses(hidden_neurons[0:y], ind_hidden_neurons[0:yz_ind], '''
    ''', on_pre='''
        v += 0.55 * volt
    ''')
    syn_hidden_output.connect(i=hidden_range, j=[k for k in range(0, yz_ind)])


    stim_tau_output = SpikeGeneratorGroup(yz_ind, indices=[k for k in range(0, yz_ind)], times = taus2 * ms)

    syn_tau_output = Synapses(stim_tau_output, ind_hidden_neurons[0:yz_ind], '''
    ''', on_pre='''
        v += 0.55 * volt
    ''')
    syn_tau_output.connect(j='i')

    output_neurons = NeuronGroup(z,
    '''dv/dt = -v/ decay_rate : volt 
         
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    output_neurons.v = 0 * volt

    output_out_range = []
    for i in range(0, y):
        for j in range(0, z):
            output_out_range.append(j)

    syn_output = Synapses(ind_hidden_neurons[0:yz_ind], output_neurons[0:z], '''
    ''', on_pre='''
        v += 1.2 * volt
    ''')

    syn_output.connect(i=[k for k in range(0, yz_ind)], j=output_out_range)

    

    mon = StateMonitor(input_neurons, 'v', record=True, dt=0.01*ms)
    M1 = StateMonitor(ind_input_neurons, 'v', record=True, dt=0.01*ms)

    M2 = StateMonitor(hidden_neurons, 'v', record=True, dt=0.01*ms)

    M3 = StateMonitor(ind_hidden_neurons, 'v', record=True, dt=0.01*ms)
    M4 = StateMonitor(output_neurons, 'v', record=True, dt=0.01*ms)

    spikemon = SpikeMonitor(input_neurons)
    spikemon_1 = SpikeMonitor(ind_input_neurons)
    spikemon_2 = SpikeMonitor(hidden_neurons)
    spikemon_3 = SpikeMonitor(ind_hidden_neurons)
    spikemon_4 = SpikeMonitor(output_neurons)

    run(10*ms)

    # # Plot v
    # figure(figsize=(10, 6))
    # for i in range(0, yz_ind): 
    #     #plot(mon.t/ms, mon.v[i], label=f'Neuron {i}')
    #     plot(M2.t/ms, M2.v[i], label=f'Neuron {i}')
    #     plot(M4.t/ms, M4.v[i], label=f'output neruon')
    # # for i in range(0, z): 
    # #     plot(M4.t/ms, M2.v[i], label=f'Neuron output')
    # xlabel('Time (ms)')
    # ylabel('Membrane potential')
    # legend()
    # title('SNN Spike Propagation Across All Layers')
    # show()

    results = []
    results.append(spikemon.spike_trains())
    results.append(spikemon_1.spike_trains())
    results.append(spikemon_2.spike_trains())
    results.append(spikemon_3.spike_trains())
    results.append(spikemon_4.spike_trains())
    

    # l_of_s = []
    # for i in range(0,x):
    #     times = spikemon.spike_trains()[i]
    #     results.append(times)
    #     if len(times) > 0:
    #         formatted_times = [f"{t/ms:.3f} ms" for t in times]
    #         print(f"Inputs N {i} spike times: {formatted_times}")

    # for i in range(0,xy_ind):
    #     times = spikemon_1.spike_trains()[i]
    #     results.append(times)
    #     if len(times) > 0:
    #         formatted_times = [f"{t/ms:.3f} ms" for t in times]
    #         print(f"Neuron input ind {i} spike times: {formatted_times}")

    # for i in range(0, y):
    #     times = spikemon_2.spike_trains()[i]
    #     results.append(times)
    #     if len(times) > 0:
    #         formatted_times = [f"{t/ms:.3f} ms" for t in times]
    #         print(f"Neuron hidden {i} spike times: {formatted_times}")

    # for i in range(0, yz_ind):
    #     times = spikemon_3.spike_trains()[i]
    #     results.append(times)
    #     if len(times) > 0:
    #         formatted_times = [f"{t/ms:.3f} ms" for t in times]
    #         print(f"Neuron hidden  idn {i} spike times: {formatted_times}")
    
    # for i in range(0, z):
    #     times = spikemon_4.spike_trains()[i]
    #     results.append(times)
    #     if len(times) > 0:
    #         formatted_times = [f"{t/ms:.3f} ms" for t in times]
    #         print(f"Neuron output  {i} spike times: {formatted_times}")
    #print(results)
    return results   # check the dictionalry apect of this as the odder might be changing so harder wirinign i-j such relationship



inputs = [1.0]
y = 2
z = 2

h_neurons_sumed_movements = [0] * y 

taus1 = [1.1, 1.1] 
taus2 = [1.2, 1.3, 1.35, 1.4] 


INFO       Cache size for target 'cython': 2347 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the 'C:\Users\irtho\.cython\brian_extensions' directory. [brian2]


In [2]:
# raw_results = run_Ymir(inputs, y, z, taus1, taus2)

# recorded_spikes = [
#     {neuron_idx: (spike_times / ms).tolist()  # convert to float ms
#      for neuron_idx, spike_times in group.items()}
#     for group in raw_results
# ]

# print("Recorded spikes:", recorded_spikes)

# for i in range(len(recorded_spikes)):
#     print(f"Group {i} spikes:")
#     for neuron_idx, spike_times in recorded_spikes[i].items():
#         print(f"  Neuron {neuron_idx}: {spike_times}")
#     print()



In [3]:
# funcition for training taus
def see_system_run():
    raw_results = run_Ymir(inputs, y, z, taus1, taus2)

    recorded_spikes = [
        {neuron_idx: (spike_times / ms).tolist()  # convert to float ms
        for neuron_idx, spike_times in group.items()}
        for group in raw_results
    ]

    print("Recorded spikes:", recorded_spikes)

    for i in range(len(recorded_spikes)):
        print(f"Group {i} spikes:")
        for neuron_idx, spike_times in recorded_spikes[i].items():
            print(f"  Neuron {neuron_idx}: {spike_times}")
        print()



def tau_shifer(spike, tau, i, strength, closer=True):
    global h_neurons_sumed_movements
    # i is equal to what syn we are on 0 and 2 are n0 and 1 & 3 --> n1
    delta = spike - tau
    j = int(i > 1)

    if closer:
        # Original behavior: bigger delta → bigger step toward spike
        step = delta * strength
        if tau + step < 0:
            h_neurons_sumed_movements[j] += 0.02
            return 0.02
        h_neurons_sumed_movements[j] += step
        return tau + step
    else:
        # Inverse scaling: bigger delta → smaller step
        if delta == 0:
            h_neurons_sumed_movements[j] += 0.02
            return tau + 0.02  # struck will always fire if - is in range? - make sure 0 is always out of range for possilbe inputs? 
        step = (strength / abs(delta)) * delta  # sign from delta
        if tau - step < 0:
            h_neurons_sumed_movements[j] += 0.02
            return 0.02
        h_neurons_sumed_movements[j] -= step
        return tau - step
        
def directions(output_spikes, desired):
     # just firinng for a single spike or not change change later
    mods = [-1] * len(desired)  # assuming we want to modify all neurons
    
    for i in range(len(desired)): 
        if desired[i] == True:    # if we wanted a spike
            if output_spikes[i] == []: # if no spike saw
                mods[i] = 2 
            else:                   # if spike saw
                mods[i] = 0
        else:
            if output_spikes[i] == []: # if we didnt want a spike and it didnt spike
                mods[i] = 3
            else:   # if we didnt want it to spike but it did
                mods[i] = 1
    return mods     


def train_taus_last_layer(inputs, taus1, taus2, strength, desired):

    # run it and gets data with current values
    raw_results = run_Ymir(inputs, y, z, taus1, taus2)

    recorded_spikes = [
        {neuron_idx: (spike_times / ms).tolist()  # convert to float ms
        for neuron_idx, spike_times in group.items()}
        for group in raw_results
    ]
     # here we have the data of all the spikes and taus and desired we send into to get new taus
    
    d = directions(recorded_spikes[2], desired)  # get the directions for the taus
    
    #print(f"directions: {d}")

    # have function for sorting and making bools for such


    # basied on direction get proper assocaltion for each direction and strength for updating and send each though tau shifter
    move = False
    count = 0
    index = 0
    for i in range((len(taus2))):
        # if spike as well
            #if i % 2 == 0: # even index taus        # will need to change to extend to multi later
        if d[index] % 2 == 0:  # 0 and 2 here
            move = True
        else:
            move = False
        if recorded_spikes[2][count] != []:  # if we saw a spike
            # print("count:", count)
            # print(i)
            # print("taus ", taus2[i])
            # if recorded_spikes[3][count][0] != []:
            #     print(f"{i}")
            #     print("spike at: ", recorded_spikes[3][count][0])
            #     print("taus2: ", taus2[i])
            #     print("strength: ", strength)
            #     print("move: ", move)
            new_tau = tau_shifer(recorded_spikes[2][count][0], taus2[i], i, strength, closer=move)
            #print("new tay and old tau: ", new_tau, taus2[i])
            taus2[i] = new_tau
    
        else: 
            #new_tau = tau_shifer(0, taus2[i], strength, closer=move)  # if no spike saw then just use 0
            print("condition of change on condition of hidden not having spike ")# DID YOU EVER GOT HEREEEE")
            #print("no spike saw so no update?" )#new tay and old tau: ", new_tau, taus2[i])
            #taus2[i] = new_tau

        if (i+1) % 2 == 0:
            count += 1
        if index == 0:
            index += 1
        elif index == 1:
            index = 0
    

    return #recorded_spikes[3]  # return the last layer spikes for now

    

In [8]:
# example 2 varying inputs to train specifics 1.0 goes to T/F and 1.5 goes to F/T 

y = 2
z = 2
taus1 = [1.1, 1.1] 
taus2 = [1.2, 1.3, 1.35, 1.4] 

h_neurons_sumed_movements = [0] * y 

# see_system_run()

for i in range(2):
    train_taus_last_layer([1.0], taus1, taus2, .1, [True, False])
    train_taus_last_layer([1.5], taus1, taus2, .1, [False, True])

# inputs = [1.0]
# see_system_run() # should only see first one spike
# inputs = [1.5]
# see_system_run()

print(h_neurons_sumed_movements)


[0.009500000000000019, -0.038000000000000006]


In [5]:
# # example 2 varying inputs to train specifics 1.0 goes to T/F and 1.5 goes to F/T 

# y = 2
# z = 2
# taus1 = [1.1, 1.1] 
# taus2 = [1.2, 1.3, 1.35, 1.4] 

# see_system_run()

# for i in range(20):
#     train_taus_last_layer([1.0], taus1, taus2, .1, [True, False])
#     train_taus_last_layer([1.5], taus1, taus2, .1, [False, True])

# inputs = [1.0]
# see_system_run() # should only see first one spike
# inputs = [1.5]
# see_system_run()

In [6]:
# example 1 single input to desired output # works for all types of desired output T/F - 4 combinations
see_system_run()

for i in range(20):
    train_taus_last_layer(inputs, taus1, taus2, .1, [True, False])

see_system_run()

Recorded spikes: [{0: [1.01]}, {0: [1.11], 1: [1.11]}, {0: [1.12], 1: [1.12]}, {0: [1.1300000000000001], 1: [1.52], 2: [1.1300000000000001], 3: [1.6]}, {0: [1.1400000000000001], 1: [1.53, 1.61]}]
Group 0 spikes:
  Neuron 0: [1.01]

Group 1 spikes:
  Neuron 0: [1.11]
  Neuron 1: [1.11]

Group 2 spikes:
  Neuron 0: [1.12]
  Neuron 1: [1.12]

Group 3 spikes:
  Neuron 0: [1.1300000000000001]
  Neuron 1: [1.52]
  Neuron 2: [1.1300000000000001]
  Neuron 3: [1.6]

Group 4 spikes:
  Neuron 0: [1.1400000000000001]
  Neuron 1: [1.53, 1.61]

Recorded spikes: [{0: [1.01]}, {0: [1.11], 1: [1.11]}, {0: [1.12], 1: [1.12]}, {0: [1.1300000000000001], 1: [], 2: [1.1300000000000001], 3: []}, {0: [1.1400000000000001], 1: []}]
Group 0 spikes:
  Neuron 0: [1.01]

Group 1 spikes:
  Neuron 0: [1.11]
  Neuron 1: [1.11]

Group 2 spikes:
  Neuron 0: [1.12]
  Neuron 1: [1.12]

Group 3 spikes:
  Neuron 0: [1.1300000000000001]
  Neuron 1: []
  Neuron 2: [1.1300000000000001]
  Neuron 3: []

Group 4 spikes:
  Neuron 

In [7]:
int(3 > 1)


1